In [1]:
#For model data
import sys
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

#for saving output
import csv
import os
import pickle

#for calculating time difference
from datetime import datetime, timedelta

from scipy.interpolate import RegularGridInterpolator

#Tools for evaraging model data
sys.path.append(r'../tools')

In [26]:
import xarray as xr

# Ensure that you haven't defined a variable or object named 'open_dataset' elsewhere in your code

# Open the dataset
url = "https://thredds.met.no/thredds/dodsC/fou-hi/barents_eps_surface/2023/03/21/T00Z/barents_sfc_20230321T00Zm21.nc"
data_barents = xr.open_dataset(url, decode_times=False)


In [28]:
data_barents.time[36]

<xarray.DataArray 'time' ()>
array(9.96920997e+36)
Coordinates:
    time     float64 9.969e+36
Attributes:
    long_name:      time since initialization
    units:          seconds since 1970-01-01 00:00:00
    calendar:       gregorian
    field:          time, scalar, series
    axis:           T
    standard_name:  time
    _ChunkSizes:    1

In [6]:
dataset0 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/metusers/johannesro/eps/barents_eps_20230211T00Z.nc")
dataset1 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/metusers/johannesro/eps/barents_eps_20230211T06Z.nc")
dataset2 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/metusers/johannesro/eps/barents_eps_20230211T12Z.nc")
dataset3 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/metusers/johannesro/eps/barents_eps_20230211T18Z.nc")
dataset4 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/metusers/johannesro/eps/barents_eps_20230213T00Z.nc")
dataset5 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/metusers/johannesro/eps/barents_eps_20230213T12Z.nc")
dataset6 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/metusers/johannesro/eps/barents_eps_20230213T18Z.nc")
dataset7 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/metusers/johannesro/eps/barents_eps_20230214T00Z.nc")
dataset8 = xr.open_dataset("https://thredds.met.no/thredds/dodsC/metusers/johannesro/eps/barents_eps_20230214T06Z.nc")

In [7]:
dataset0

<xarray.Dataset>
Dimensions:                  (ensemble_member: 6, s_rho: 42, s_w: 43, time: 67,
                              X: 739, Y: 949, depth: 1)
Coordinates:
  * time                     (time) datetime64[ns] 2023-02-11 ... 2023-02-13T...
  * X                        (X) float32 2.786e+05 2.811e+05 ... 2.124e+06
  * Y                        (Y) float32 -8.979e+05 -8.954e+05 ... 1.472e+06
  * depth                    (depth) float64 0.0
  * ensemble_member          (ensemble_member) int32 0 1 2 3 4 5
    lon                      (Y, X) float64 ...
    lat                      (Y, X) float64 ...
Dimensions without coordinates: s_rho, s_w
Data variables: (12/21)
    hc                       (ensemble_member) float64 ...
    Cs_r                     (ensemble_member, s_rho) float64 ...
    Cs_w                     (ensemble_member, s_w) float64 ...
    forecast_reference_time  (ensemble_member) datetime64[ns] ...
    projection_lambert       (ensemble_member) int32 ...
    h                        (ensemble_member, Y, X) float64 ...
    ...                       ...
    Uwind                    (time, ensemble_member, Y, X) float32 ...
    Vwind                    (time, ensemble_member, Y, X) float32 ...
    ice_concentration        (time, ensemble_member, Y, X) float32 ...
    ice_thickness            (time, ensemble_member, Y, X) float32 ...
    ice_u                    (time, ensemble_member, Y, X) float32 ...
    ice_v                    (time, ensemble_member, Y, X) float32 ...
Attributes: (12/33)
    file:                            /lustre/storeB/project/fou/hi/oper/baren...
    Conventions:                     CF-1.4, SGRID-0.3
    type:                            ROMS/TOMS history file
    title:                           Barents-2.5km - ROMS
    var_info:                        /home/havis/sea/ROMS/metroms_apps/barent...
    rst_file:                        /lustre/storeB/project/fou/hi/oper/baren...
    ...                              ...
    ana_file:                        ROMS/Functionals/ana_btflux.h, ROMS/Func...
    CPP_options:                     BARENTS, ADD_FSOBC, ADD_M2OBC, ALBEDO, A...
    DAinfo_EnKF:                     \nDA_DAY            "6567.0 days since 1...
    history:                         Sat Feb 11 08:09:18 2023: ncpdq -O -a ti...
    NCO:                             netCDF Operators version 4.7.9 (Homepage...
    DODS_EXTRA.Unlimited_Dimension:  time

#### Variant with rolling evarage of 24 ensembles for the whole forecasting period with 6 hour ensembles update.
Takes 4 preciding datasets for calculating mean 24 ensembles variables for the next 6 (or what is left before new update at 00,06,12,18) hours.
At time of new forecast drops the most outdated 6 ensembles and add the most recent 6.


Description of the model:


Sea ice drift forecasts were generated using a model that releases data in discrete files at four specific times daily: 00:00, 06:00, 12:00, and 18:00. Each of these files provides an hourly forecast of instantaneous sea ice drift measurements projected for the subsequent 66 hours. Crucially, every file contains data from six distinct ensemble predictions, summing to a total of 24 unique ensembles over the course of a day. The ensembles are systematically numbered from 0 to 23, with the 00:00 file encompassing ensembles 0-5, the 06:00 file including ensembles 6-11, the 12:00 file comprising ensembles 12-17, and the 18:00 file containing ensembles 18-23. At the model's finest resolution of 2.5 km, we observed that the sea ice drift is profoundly influenced by ocean surface features, which emerge as vortices in the drift data. It's worth noting that such vortices at this granularity are not commonly evident in real-world observations. Given the variability of these vortices across different ensembles, we employed an averaging technique across ensembles to decrease their influence and enhance the accuracy of our forecasts.



* From the article (also it makes sense to take some general info from Victor's article)
The sea ice model used in Barents-2.5 is the Los Alamos sea ice model (CICE), version 5.1 (Hunke et al., 2017). CICE describes both dynamic and thermodynamic processes, using a elastic-viscous-plastic (EVP) rheology (Hunke and Dukowicz, 1997, Röhrs et al, 2023). The ice velocity u is calculated from the sea ice momentum equation that account for air and water drag, Coriolis force, sea surface tilt, and the divergence of internal ice stress.

In [4]:
# Extract grid variables

X = dataset0['X']
Y = dataset0['Y']
lon = dataset0['lon']
lat = dataset0['lat']
model_proj4 = dataset0.projection_lambert.proj4

%store X
%store Y
#%store lon
#%store lat
%store model_proj4

Stored 'X' (DataArray)
Stored 'Y' (DataArray)
Stored 'model_proj4' (str)


In [5]:
datasets = [dataset0, dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8]

In [6]:
print("Ensembles")
for idx, ensemble in enumerate(datasets):
    print(f"dataset{idx}: {datasets[idx].ensemble_member.values}")
print("Start time")
for idx, ensemble in enumerate(datasets):
    print(f"dataset{idx}: {datasets[idx].time[0].values} - {datasets[idx].time[-1].values}, {len(datasets[idx].time)- 1} forecast")

Ensembles
dataset0: [0 1 2 3 4 5]
dataset1: [ 6  7  8  9 10 11]
dataset2: [12 13 14 15 16 17]
dataset3: [18 19 20 21 22 23]
dataset4: [0 1 2 3 4 5]
dataset5: [12 13 14 15 16 17]
dataset6: [18 19 20 21 22 23]
dataset7: [0 1 2 3 4 5]
dataset8: [ 6  7  8  9 10 11]
Start time
dataset0: 2023-02-11T00:00:00.000000000 - 2023-02-13T18:00:00.000000000, 66 forecast
dataset1: 2023-02-11T06:00:00.000000000 - 2023-02-14T00:00:00.000000000, 66 forecast
dataset2: 2023-02-11T12:00:00.000000000 - 2023-02-14T06:00:00.000000000, 66 forecast
dataset3: 2023-02-11T18:00:00.000000000 - 2023-02-14T12:00:00.000000000, 66 forecast
dataset4: 2023-02-13T00:00:00.000000000 - 2023-02-15T18:00:00.000000000, 66 forecast
dataset5: 2023-02-13T12:00:00.000000000 - 2023-02-16T06:00:00.000000000, 66 forecast
dataset6: 2023-02-13T18:00:00.000000000 - 2023-02-16T12:00:00.000000000, 66 forecast
dataset7: 2023-02-14T00:00:00.000000000 - 2023-02-16T18:00:00.000000000, 66 forecast
dataset8: 2023-02-14T06:00:00.000000000 - 2023-

In [36]:
from model_data_prep import round_start_time, round_end_time, time_difference

# SAR images timestamps
t_sar1 = '2023-02-12T08:01:51'
t_sar2 = '2023-02-14T07:45:31'

# Rounding the SAR timestamps to align with the nearest whole hour of model timestamps
t_start = round_start_time(t_sar1)
t_end = round_end_time(t_sar2)

print(f'SAR1 time is {t_sar1}, Model start time for the time period is {t_start}')
print(f'SAR2 time is {t_sar2}, Model end time for the time period is {t_end}')


# Extracting the model time variable data corresponding to the time period between the rounded start and end times
time_period_barents = dataset3.time.sel(time=slice(t_start,t_end))

print(f'The duration of the time period is {len(time_period_barents)-1} whole hours')

SAR1 time is 2023-02-12T08:01:51, Model start time for the time period is 2023-02-12T08:00:00
SAR2 time is 2023-02-14T07:45:31, Model end time for the time period is 2023-02-14T08:00:00
The duration of the time period is 48 whole hours


In [37]:
time_period_barents

<xarray.DataArray 'time' (time: 49)>
array(['2023-02-12T08:00:00.000000000', '2023-02-12T09:00:00.000000000',
       '2023-02-12T10:00:00.000000000', '2023-02-12T11:00:00.000000000',
       '2023-02-12T12:00:00.000000000', '2023-02-12T13:00:00.000000000',
       '2023-02-12T14:00:00.000000000', '2023-02-12T15:00:00.000000000',
       '2023-02-12T16:00:00.000000000', '2023-02-12T17:00:00.000000000',
       '2023-02-12T18:00:00.000000000', '2023-02-12T19:00:00.000000000',
       '2023-02-12T20:00:00.000000000', '2023-02-12T21:00:00.000000000',
       '2023-02-12T22:00:00.000000000', '2023-02-12T23:00:00.000000000',
       '2023-02-13T00:00:00.000000000', '2023-02-13T01:00:00.000000000',
       '2023-02-13T02:00:00.000000000', '2023-02-13T03:00:00.000000000',
       '2023-02-13T04:00:00.000000000', '2023-02-13T05:00:00.000000000',
       '2023-02-13T06:00:00.000000000', '2023-02-13T07:00:00.000000000',
       '2023-02-13T08:00:00.000000000', '2023-02-13T09:00:00.000000000',
       '2023-02-13T10:00:00.000000000', '2023-02-13T11:00:00.000000000',
       '2023-02-13T12:00:00.000000000', '2023-02-13T13:00:00.000000000',
       '2023-02-13T14:00:00.000000000', '2023-02-13T15:00:00.000000000',
       '2023-02-13T16:00:00.000000000', '2023-02-13T17:00:00.000000000',
       '2023-02-13T18:00:00.000000000', '2023-02-13T19:00:00.000000000',
       '2023-02-13T20:00:00.000000000', '2023-02-13T21:00:00.000000000',
       '2023-02-13T22:00:00.000000000', '2023-02-13T23:00:00.000000000',
       '2023-02-14T00:00:00.000000000', '2023-02-14T01:00:00.000000000',
       '2023-02-14T02:00:00.000000000', '2023-02-14T03:00:00.000000000',
       '2023-02-14T04:00:00.000000000', '2023-02-14T05:00:00.000000000',
       '2023-02-14T06:00:00.000000000', '2023-02-14T07:00:00.000000000',
       '2023-02-14T08:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2023-02-12T08:00:00 ... 2023-02-14T08:00:00
Attributes:
    long_name:      time since initialization
    field:          time, scalar, series
    axis:           T
    standard_name:  time
    _ChunkSizes:    1

In [38]:
# Store variables
time_diff_start, time_diff_end, total_time_diff = time_difference(t_sar1, t_sar2, time_period_barents)

%store time_diff_start
%store time_diff_end
%store total_time_diff
%store time_period_barents

Time difference between SAR1 and the start of the model time period is 111 seconds (1.85 minutes).
Time difference between SAR2 and the end of the model time period is 869 seconds (14.483 minutes).
Total time difference between SAR1 and SAR2 images is 171820 seconds (1.989 days).
Stored 'time_diff_start' (int64)
Stored 'time_diff_end' (int64)
Stored 'total_time_diff' (int64)
Stored 'time_period_barents' (DataArray)


Methodology: Rolling Averaging of Ensemble Ice Drift Velocities

The goal of this methodology is to compute the average ice drift velocities, specifically ice_u and ice_v, over a designated time range using a sliding window of ensemble forecasts. This approach seeks to always use the latest available ensemble forecasts to provide an updated and more accurate representation of ice drift velocities.

Defining the Forecast Time Range:
The start and end times of the forecast period are specified. The period is then divided into hourly intervals using the pd.date_range() function, producing a sequence of timestamps.

Selection of Relevant Datasets:
For each hourly timestamp:

All datasets that have forecast data encompassing the current timestamp are identified.
These datasets are sorted based on their starting forecast time.
The four most recent datasets are selected to ensure the usage of the latest ensemble forecasts.
Data Extraction and Concatenation:
For each of the selected datasets, the ice_u and ice_v data corresponding to the current timestamp are extracted. Additionally, a spatial subset of the data is chosen based on specified row (r) and column (c) indices. To handle missing data, any NaN values are retained by using the fillna(np.nan) function. The extracted ensemble data from all four datasets are concatenated along a new dimension named ensemble_member, resulting in a combined dataset with data from 24 ensemble members.

Averaging Over Ensembles:
The concatenated ensemble data are then averaged over the ensemble_member dimension, yielding a single averaged value for ice_u and ice_v for each hourly timestamp.

Creation of Time Series DataArrays:
The hourly averaged ice_u and ice_v values are concatenated along the time dimension, producing two continuous time series DataArray objects that represent the average ice drift velocities over the entire forecast period.

The rolling_avg_24_ensembles function implements the aforementioned methodology, taking as input a list of ensemble datasets and the desired start and end times of the forecast period. It returns two DataArray objects containing the averaged ice drift velocities (avg_ice_u_da and avg_ice_v_da) for the entire period. This approach ensures a continuous and updated representation of the average ice drift velocities by always considering the latest ensemble forecasts.

Short version:
To provide a continuous and updated representation of ice drift velocities, we employed a rolling averaging approach. For each hour within a designated forecast period, we sourced data from the four most recent ensemble forecast datasets available at that time. By prioritizing the latest ensemble forecasts, we ensured the inclusion of the most up-to-date predictions in our analysis. For each hourly timestep, the ice drift velocities were extracted from these datasets, spatially subsetted, and then averaged across all 24 ensemble members. This process generated a time series of average ice drift velocities spanning the entire forecast period.

##### Calculating rolling average for full and subset gtids

In [23]:
from model_data_prep import rolling_avg_24_ensembles

In [24]:
# For the whole extent
avg_ice_u, avg_ice_v = rolling_avg_24_ensembles(datasets, t_start, t_end, subset = False)

##### Calculate mean hourly (NOT cumulative) drift

In [ ]:
from model_data_prep import non_cumulative_ice_displacement

In [ ]:
xx, yy, hourly_mean_disp_u, hourly_mean_disp_v =  non_cumulative_ice_displacement(X, Y, avg_ice_u, avg_ice_v, time_period_barents, time_diff_start, time_diff_end)

##### Saving speed as pickle files

In [ ]:
pickle_path_u = "/home/jovyan/experiment_data/2022-2023_48h_experiment/drift_output/14_16_feb/barents_24ens_14_16feb_hourly_ice_u.pkl"
pickle_path_v = "/home/jovyan/experiment_data/2022-2023_48h_experiment/drift_output/14_16_feb/barents_24ens_14_16feb_hourly_ice_v.pkl"
# Save the list
with open(pickle_path_u, "wb") as f:
    pickle.dump(hourly_mean_disp_u, f)
    
with open(pickle_path_v, "wb") as f:
    pickle.dump(hourly_mean_disp_v, f)

##### Calculating cummulative (integrated) drift for the full extent

In [26]:
from model_data_prep import cumulative_ice_displacement

In [27]:
# Calculate cumulative drift for the whole grid 
xx_b, yy_b, cum_dx_b, cum_dy_b = cumulative_ice_displacement(X, Y, avg_ice_u, avg_ice_v, time_period_barents, time_diff_start, time_diff_end)

1 hour done
2 hour done
3 hour done
4 hour done
5 hour done
6 hour done
7 hour done
8 hour done
9 hour done
10 hour done
11 hour done
12 hour done
13 hour done
14 hour done
15 hour done
16 hour done
17 hour done
18 hour done
19 hour done
20 hour done
21 hour done
22 hour done
23 hour done
24 hour done
25 hour done
26 hour done
27 hour done
28 hour done
29 hour done
30 hour done
31 hour done
32 hour done
33 hour done
34 hour done
35 hour done
36 hour done
37 hour done
38 hour done
39 hour done
40 hour done
41 hour done
42 hour done
43 hour done
44 hour done
45 hour done
46 hour done
47 hour done
48 hour done


##### Saving cum displacements as pickle files

In [ ]:
pickle_path_u = "/home/jovyan/experiment_data/2022-2023_48h_experiment/drift_output/14_16_feb/barents_24ens_14_16feb_cum_ice_u.pkl"
pickle_path_v = "/home/jovyan/experiment_data/2022-2023_48h_experiment/drift_output/14_16_feb/barents_24ens_14_16feb_cum_ice_v.pkl"
# Save the list
with open(pickle_path_u, "wb") as f:
    pickle.dump(cum_dx_b, f)
    
with open(pickle_path_v, "wb") as f:
    pickle.dump(cum_dy_b, f)

##### Calculating cummulative (integrated) drift for the subset extent

In [16]:
#Import variables for extracting subsets from SAR1-SAR2_drift.ipynb
%store -r min_row
%store -r max_row
%store -r min_col
%store -r max_col
%store -r X_subset
%store -r Y_subset
%store -r lon_subset
%store -r lat_subset

%store -r lon
%store -r lat

In [7]:
barents_model_drift_path = "/home/jovyan/experiment_data/2022-2023_48h_experiment/drift_output/12_14_feb/barents_24ens_cum_disp_14-16feb.csv"
with open(barents_model_drift_path) as csvfile:
    csvreader = csv.reader(csvfile)
    # check if the file has a header row
    has_header = csv.Sniffer().has_header(csvfile.read(1024))
    csvfile.seek(0)  # reset file pointer
    #skip header row if it exists
    if has_header:
        next(csvreader)
    displacements = np.array([row for row in csvreader]).astype(float)
    cum_dx_b = displacements[:, [2]] 
    cum_dy_b = displacements[:, [3]]

In [11]:
full_model_shape = (949, 739) #lon.shape
cum_dx_b_total = np.reshape(cum_dx_b, full_model_shape)
cum_dy_b_total = np.reshape(cum_dy_b, full_model_shape)

In [17]:
cum_dx_b_subset = cum_dx_b_total[min_row:max_row+1, min_col:max_col+1]
cum_dy_b_subset = cum_dy_b_total[min_row:max_row+1, min_col:max_col+1]
lon_subset = lon[min_row:max_row+1, min_col:max_col+1]
lat_subset = lat[min_row:max_row+1, min_col:max_col+1]

In [18]:
cum_dy_b_total.shape, cum_dy_b_subset.shape

((949, 739), (283, 219))

In [21]:
from osgeo import osr
# Define the target spatial reference (WGS 84)
target_srs = osr.SpatialReference()
target_srs.ImportFromEPSG(4326)

# Define the source spatial reference (model's projection)
model_srs = osr.SpatialReference()
model_srs.ImportFromProj4(model_proj4)

# Create the coordinate transformation
transformation = osr.CoordinateTransformation(model_srs, target_srs)

# Calculate teh final coordinates x2, y2

x1, y1 = np.meshgrid(X_subset, Y_subset)
print(x1.shape, y1.shape)
x1, y1 = x1.flatten(), y1.flatten()
print(x1.shape, y1.shape)
dx = cum_dx_b_subset.flatten()
dy = cum_dy_b_subset.flatten()

x2 = x1 + dx
y2 = y1 + dy

# Stack the coordinates for transformation
points = np.stack((x2, y2), axis=1)
# Apply the coordinate transformation
transformed_points = transformation.TransformPoints(points)


# Extract longitude and latitude
lat2_subset = np.array([point[0] for point in transformed_points])
lon2_subset = np.array([point[1] for point in transformed_points])

(283, 219) (283, 219)
(61977,) (61977,)


In [ ]:
# Need to change the function (add r and c to make it work via imported function)
# For the subset if provided
avg_ice_u_subset, avg_ice_v_subset = rolling_avg_24_ensembles(datasets, t_start, t_end, subset = True)

In [ ]:
# Calculate cumulative drift for subset grid 
xx_b_subset, yy_b_subset, cum_dx_b_subset, cum_dy_b_subset = cumulative_ice_displacement(X_subset,Y_subset, avg_ice_u_subset, avg_ice_v_subset, time_period_barents, time_diff_start,time_diff_end)

In [ ]:
#Store variables
%store xx_b
%store yy_b
%store cum_dx_b
%store cum_dy_b 

'''
%store xx_b_subset
%store yy_b_subset
%store cum_dx_b_subset
%store cum_dy_b_subset 
'''

##### Calculating average concentration for t[0]

In [10]:
dataset0.ensemble_member.values

array([0, 1, 2, 3, 4, 5], dtype=int32)

In [ ]:
def avg_concentration(datasets, time_period):

    available_datasets = [ds for ds in datasets if (ds.time[0].values <= time_period[0])]
    print(f'{len(available_datasets)} avalible datasets')
    conc_t_start_ens = xr.concat([ds.sel(time=time_period[0]).ice_concentration.fillna(np.nan) for ds in available_datasets], dim='ensemble_member')
    print(f'ensembles for averaging are {conc_t_start_ens.ensemble_member.values}')
    avg_conc_t_start = conc_t_start_ens.mean(dim='ensemble_member')
    
    return avg_conc_t_start

In [ ]:
avg_conc_t_start = avg_concentration(datasets, time_period_barents)

In [ ]:
feb1214_conc = avg_conc_t_start
%store feb1214_conc

In [ ]:
avg_conc_t_start.plot()


##### Export cumulative drift to csv  (For chalmers)
(alternative way is just to pass it as stored variables)

In [24]:
# Subset grid
output_dir_subset_comp = r'/home/jovyan/experiment_data/2022-2023_48h_experiment/drift_output/12_14_feb/'

lon1 = lon_subset.data.flatten()
lat1 = lat_subset.data.flatten()

points=zip(lon1, lat1, lon2_subset, lat2_subset)
header=['lon1','lat1', 'lon2','lat2']

#file_name = f'24ens_roll_cum_displacement_{filename_t_start}-{filename_t_end}.csv'
file_name = f'barents_model_24ens_cum_disp_for_chalmers.csv'
file_path = os.path.join(output_dir_subset_comp, file_name)

with open(file_path, 'w', newline='') as csv_file:
            out = csv.writer(csv_file, delimiter=',')
            out.writerow(header)
            out.writerows(points)

In [ ]:
# full grid
output_dir_subset_comp =  r'/home/jovyan/experiment_data/2022-2023_48h_experiment/drift_output/14_16_feb'

lon1 = lon.data.flatten()
lat1 = lat.data.flatten()

points=zip(lon1, lat1, cum_dx_b[-1], cum_dy_b[-1])
header=['lon1','lat1', 'du','dv']

file_name = f'barents_24ens_cum_disp_14-16feb.csv'
file_path = os.path.join(output_dir_subset_comp, file_name)

with open(file_path, 'w', newline='') as csv_file:
            out = csv.writer(csv_file, delimiter=',')
            out.writerow(header)
            out.writerows(points)

In [ ]:
# Subset grid
output_dir_subset_comp =  r'/home/jovyan/data/model_data_process_output/for_comparison/subset_comparison_with_sar'

lon1 = lon_subset.data.flatten()
lat1 = lat_subset.data.flatten()

points=zip(lon1, lat1, cum_dx_b_subset[-1], cum_dy_b_subset[-1])
header=['lon1','lat1', 'du','dv']

#file_name = f'24ens_roll_cum_displacement_{filename_t_start}-{filename_t_end}.csv'
file_name = f'barents_subset_24ens_cum_displacement.csv'
file_path = os.path.join(output_dir_subset_comp, file_name)

with open(file_path, 'w', newline='') as csv_file:
            out = csv.writer(csv_file, delimiter=',')
            out.writerow(header)
            out.writerows(points)

In [ ]:
# Export for Chalmers forecasting (with buffer)

'''
need to add but it is not needed for cumulative drift
'''

##### Calculation of variations for estimation of realibility of instantanious values 

In [ ]:
total_velocity_roll = np.sqrt(avg_ice_u_subset**2 + avg_ice_v_subset**2)
total_velocity_roll.name = "total_velocity"
total_velocity_roll[0].plot()

In [ ]:
velocity_variation_roll = total_velocity_roll.std(dim='time')
velocity_variation_roll.plot()

In [ ]:
mean_velocity_roll = total_velocity_roll.mean(dim='time')
CV_roll = (velocity_variation_roll / mean_velocity_roll) * 100
CV_roll.plot(vmin=0, vmax=200)

Interesting the way rolling averaging provide less variation for gradient ice drift areas and more variation within vortecies..that makes sence that vortecies change a lot from ensemble to ensemble and every 6 hour. It probably means that they not that connected for one ensemble after 24 hours.  